# Recommender System

## Collaborative Filtering

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

### Importing PySpark

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.1.1-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('recommender_system').getOrCreate()

### Importing Data

In [4]:
df = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

## Exploratory Data Analysis

In [7]:
df.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



### Shape of the dataset

In [8]:
print((df.count(),len(df.columns)))

(1501, 3)


In [9]:
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



### Missing Values

In [10]:
from pyspark.sql.functions import isnan, when, count, col

In [11]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|     0|     0|
+-------+------+------+



In [12]:
df.columns

['movieId', 'rating', 'userId']

In [13]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



## Model Building

### Train Test Split

We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [14]:
train_data , test_data = df.randomSplit([0.8,0.2])

### ALS Recommender Algorithm

In [15]:
from pyspark.ml.recommendation import ALS

In [16]:
#Default as per documentation
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [18]:
model = als.fit(train_data)

In [19]:
predictions = model.transform(test_data)

In [21]:
predictions.show(5)

#eg- rating is 1 but my model predicted rating of 2

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26| 2.0119586|
|     31|   1.0|    13| 1.9506198|
|     85|   2.0|    20| 2.7528925|
|     85|   1.0|    15| 1.4837852|
|     85|   5.0|     8| 3.9929185|
+-------+------+------+----------+
only showing top 5 rows



### Evaluation

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

In [24]:
#Comparing the rating column with predicted column
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [25]:
rmse = evaluator.evaluate(predictions)

In [26]:
print('RMSE')
print(rmse)

#Not great due to less data

RMSE
1.872653449708343


### Recommendation for Specific User

In [28]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId','userId'])

In [29]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     37|    11|
|     39|    11|
|     48|    11|
|     66|    11|
|     77|    11|
|     81|    11|
|     82|    11|
|     97|    11|
+-------+------+



In [30]:
recommendations = model.transform(single_user)

In [31]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     77|    11|   4.991973|
|     48|    11|  1.8184073|
|     66|    11|  1.8100948|
|     81|    11|  1.7049825|
|     82|    11|  1.3522993|
|     97|    11|  1.3498913|
|     37|    11|  0.8737135|
|     39|    11|-0.65319556|
+-------+------+-----------+



**This means that userId 11 will like the movieID 77 very much as the predicted rating is almost 5 and so on for other movies**